In [1]:
from underdog import info

## Overview
**Status**: Development

**Tested**: Windows only

**Requires**: Python 3.8+

The *underdog* package provides Python classes and functions to easily access free equity-related data via Panda DataFrames. Data is automatically downloaded, kept up to date, and cached on disk for convenience and to improve performance and limit API requests to the data providers (which often rate limit the APIs). Currently, data is pulled from [Polygon](https://polygon.io), [finviz](https://finviz.com), and TD Ameritrade. 

Feel free to send questions, comments or feedback to: nanoonan at marvinsmind dot com.

## Installation

1. Install [parkit](https://github.com/nanoonan/parkit), which provides the storage layer. 

2. Install _underdog_ by opening a command prompt, navigating to the top-level directory of the git installation, and running the following command.

```
python -m pip install .
```

3. You'll need a free account with [Polygon](https://polygon.io) and a developer account with TD Ameritrade. The documentation for [tda-api](https://tda-api.readthedocs.io/en/latest/index.html) has more information on setting up a TDA developer account.

4. Finally, set up the environment variables used to access data from your accounts. The next cell describes these variables.

In [2]:
info('environment')

╒════════════════════════╤═══════════════════════════════════════════════╕
│ Environment Variable   │ Description                                   │
╞════════════════════════╪═══════════════════════════════════════════════╡
│ POLYGON_API_KEY        │ The API key from your Polygon account.        │
├────────────────────────┼───────────────────────────────────────────────┤
│ TDA_TOKEN_PATH         │ Path to file where TDA auth token is stored.  │
├────────────────────────┼───────────────────────────────────────────────┤
│ TDA_API_KEY            │ The API key from your TDA developer account.  │
├────────────────────────┼───────────────────────────────────────────────┤
│ TDA_REDIRECT_URI       │ In most cases this will be https://localhost. │
├────────────────────────┼───────────────────────────────────────────────┤
│ TDA_ACCOUNT_ID         │ Account id for your TDA trading account.      │
╘════════════════════════╧═══════════════════════════════════════════════╛


## API

In [3]:
# The main classes for accessing historic data
info('classes')

╒══════════════════════════╤════════════════════════════════════════════════════════════════════════╕
│ Type                     │ Description                                                            │
╞══════════════════════════╪════════════════════════════════════════════════════════════════════════╡
│ [Market, Day, IntraDay]  │ The historic data classes implement a Dict-like interface to access    │
│                          │ historic stock data. Valid keys are dates, integers, or strings        │
│                          │ containing dates in 'YYYY-MM-DD' format. Slices are also supported.    │
│                          │ Assuming x is an instance of Market, Day, or IntraDay, here are some   │
│                          │ examples: x['2020-01-01'], x['2020-01-01':'2021-01-01'], x[-252:]      │
│                          │ (returns last 252 trading days of data). Data is returned in a Panda   │
│                          │ DataFrame and downloaded and kept up to date automati

In [4]:
info('functions')

╒══════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════╕
│ Function                             │ Description                                                           │
╞══════════════════════════════════════╪═══════════════════════════════════════════════════════════════════════╡
│ intraday(symbol)                     │ Returns current intraday one-minute data for the specified ticker.    │
├──────────────────────────────────────┼───────────────────────────────────────────────────────────────────────┤
│ tickers(update = False)              │ Returns table of all tickers. Set update to True when calling for the │
│                                      │ first time or to download the latest data.                            │
├──────────────────────────────────────┼───────────────────────────────────────────────────────────────────────┤
│ ticker_details(symbol)               │ Returns info about a ticker like market cap, institutio

## Examples

In [5]:
from underdog import Market
market = Market()
# Invoke this once to download 2 years of data...will take over an hour
market.update()

True

In [6]:
# Get a dataframe of last two years of daily data for all tickers
market[:]

,symbol,volume,vwap,open,close,high,low,date,twap
0,A,1609122,77.1685,76.9800,77.4200,77.4600,76.3000,2019-04-26,76.86806
1,AA,2257704,26.9973,27.0500,26.9100,27.2500,26.8400,2019-04-26,27.046952
2,AAAU,30402,12.8574,12.8200,12.8450,12.8700,12.8200,2019-04-26,12.845833
3,AABA,3279169,76.1562,76.4400,76.1200,76.6800,75.5300,2019-04-26,76.101546
4,AAC,85690,1.5970,1.5900,1.6400,1.6600,1.5250,2019-04-26,1.591701
...,...,...,...,...,...,...,...,...,...
4573296,RBIN,1,27.6500,27.7602,27.7602,27.7602,27.7602,2021-05-14,RBIN
4573297,QPT,10,24.1599,24.2188,24.2188,24.2188,24.2188,2021-05-14,QPT
4573298,RODI,100,115.0628,116.7316,116.7316,116.7316,116.7316,2021-05-14,RODI
4573299,PSMR,344,21.0786,21.1585,21.1585,21.1585,21.1585,2021-05-14,PSMR


In [7]:
from underdog import ticker_details
# Get useful information about a ticker
ticker_details('TSLA')

{'market_cap': 568110000000,
 'shares_float': 774230000,
 'shares_outstanding': 963330000,
 'insider_ownership': 0.0,
 'institutional_ownership': 0.44,
 'retail_ownership': 0.56,
 'short_float': 0.05,
 'employees': 70757,
 'has_options': True,
 'is_shortable': True}

In [8]:
from underdog import IntraDay
# Last five days of TSLA intraday three minute data
df = IntraDay('TSLA', period = 3)[-5:]
# Filter for only regular hours trading (1 = pre-market, 2 = regular hours, 3 = extended hours)
df[df['trading_segment'] == 2]

,timestamp,open,close,low,high,volume,trading_segment,twap,timeslot
71,2021-05-10 09:30:00-04:00,664.9000,660.2812,659.0100,665.0500,416090,2,661.934012,110
72,2021-05-10 09:33:00-04:00,660.1693,658.0200,658.0000,661.3300,215961,2,659.731306,111
73,2021-05-10 09:36:00-04:00,658.1900,656.7279,655.3400,658.4299,253527,2,656.850914,112
74,2021-05-10 09:39:00-04:00,656.5568,655.1838,653.4100,656.8100,287776,2,655.077686,113
75,2021-05-10 09:42:00-04:00,655.0050,656.0800,653.1200,656.8400,316319,2,654.968006,114
...,...,...,...,...,...,...,...,...,...
1422,2021-05-14 15:45:00-04:00,588.0100,589.8700,587.8400,590.3400,227810,2,589.114091,235
1423,2021-05-14 15:48:00-04:00,589.9169,589.7850,588.7513,589.9599,145852,2,589.354121,236
1424,2021-05-14 15:51:00-04:00,589.8600,590.9129,589.5000,591.1700,260416,2,590.329323,237
1425,2021-05-14 15:54:00-04:00,590.8686,590.0015,590.0000,590.9500,228326,2,590.485509,238


In [9]:
from underdog import Day
# Last 20 years of daily XOM data
df = Day('XOM')[:]
df

,open,high,low,close,volume,date,twap
0,44.100,44.720,44.050,44.530,9779400,2001-05-14,44.393036
1,44.530,44.770,44.325,44.725,9016800,2001-05-15,44.543466
2,44.725,45.000,44.520,44.750,16337000,2001-05-16,44.760015
3,44.575,44.580,44.135,44.365,11597600,2001-05-17,44.350867
4,44.475,45.125,44.400,45.100,14289200,2001-05-18,44.756795
...,...,...,...,...,...,...,...
5029,63.240,64.020,62.530,62.580,31956561,2021-05-10,63.293827
5030,61.625,62.385,60.385,60.590,34563768,2021-05-11,61.404912
5031,60.010,61.680,59.750,60.040,34426479,2021-05-12,60.715042
5032,59.000,60.450,58.750,59.300,24268233,2021-05-13,59.603531
